# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Feb 3 2023 8:30AM,256565,10,0086352838,ISDIN Corporation,Released
1,Feb 3 2023 8:30AM,256565,10,0086352845,ISDIN Corporation,Released
2,Feb 3 2023 8:30AM,256565,10,0086352842,ISDIN Corporation,Released
3,Feb 3 2023 8:30AM,256565,10,0086352846,ISDIN Corporation,Released
4,Feb 3 2023 8:30AM,256565,10,0086352844,ISDIN Corporation,Released
5,Feb 3 2023 8:30AM,256565,10,0086353054,ISDIN Corporation,Released
6,Feb 3 2023 8:30AM,256565,10,0086352847,ISDIN Corporation,Released
7,Feb 3 2023 8:30AM,256565,10,0086353060,ISDIN Corporation,Released
8,Feb 3 2023 8:30AM,256565,10,0086353055,ISDIN Corporation,Released
9,Feb 3 2023 8:30AM,256565,10,0086353061,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
36,256565,Released,21
37,256568,Released,5
38,256569,Released,1
39,256570,Executing,1
40,256574,Released,2


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
256565,NaN,NaN,21.0
256568,NaN,NaN,5.0
256569,NaN,NaN,1.0
256570,NaN,1.0,NaN
256574,NaN,NaN,2.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
256444,3.0,5.0,2.0
256454,0.0,1.0,0.0
256456,0.0,0.0,1.0
256470,0.0,3.0,0.0
256471,0.0,4.0,9.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
256444,3,5,2
256454,0,1,0
256456,0,0,1
256470,0,3,0
256471,0,4,9


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,256444,3,5,2
1,256454,0,1,0
2,256456,0,0,1
3,256470,0,3,0
4,256471,0,4,9


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,256444,3,5,2
1,256454,,1,
2,256456,,,1
3,256470,,3,
4,256471,,4,9


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Feb 3 2023 8:30AM,256565,10,ISDIN Corporation
14,Feb 3 2023 8:30AM,256564,10,ISDIN Corporation
27,Feb 3 2023 8:30AM,256568,10,Eywa Pharma Inc.
32,Feb 3 2023 8:30AM,256569,10,"Nextsource Biotechnology, LLC"
33,Feb 2 2023 4:12PM,256574,16,Sartorius Bioprocess Solutions
35,Feb 2 2023 4:02PM,256570,19,"AdvaGen Pharma, Ltd"
36,Feb 2 2023 3:28PM,256562,22,"NBTY Global, Inc."
37,Feb 2 2023 2:51PM,256557,21,"NBTY Global, Inc."
38,Feb 2 2023 2:45PM,256556,10,"Digital Brands, LLC"
39,Feb 2 2023 2:40PM,256555,19,ACG North America LLC


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Feb 3 2023 8:30AM,256565,10,ISDIN Corporation,,,21
1,Feb 3 2023 8:30AM,256564,10,ISDIN Corporation,,,6
2,Feb 3 2023 8:30AM,256568,10,Eywa Pharma Inc.,,,5
3,Feb 3 2023 8:30AM,256569,10,"Nextsource Biotechnology, LLC",,,1
4,Feb 2 2023 4:12PM,256574,16,Sartorius Bioprocess Solutions,,,2
5,Feb 2 2023 4:02PM,256570,19,"AdvaGen Pharma, Ltd",,1,
6,Feb 2 2023 3:28PM,256562,22,"NBTY Global, Inc.",,,1
7,Feb 2 2023 2:51PM,256557,21,"NBTY Global, Inc.",,,1
8,Feb 2 2023 2:45PM,256556,10,"Digital Brands, LLC",,,1
9,Feb 2 2023 2:40PM,256555,19,ACG North America LLC,,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 3 2023 8:30AM,256565,10,ISDIN Corporation,21,,
1,Feb 3 2023 8:30AM,256564,10,ISDIN Corporation,6,,
2,Feb 3 2023 8:30AM,256568,10,Eywa Pharma Inc.,5,,
3,Feb 3 2023 8:30AM,256569,10,"Nextsource Biotechnology, LLC",1,,
4,Feb 2 2023 4:12PM,256574,16,Sartorius Bioprocess Solutions,2,,
5,Feb 2 2023 4:02PM,256570,19,"AdvaGen Pharma, Ltd",,1,
6,Feb 2 2023 3:28PM,256562,22,"NBTY Global, Inc.",1,,
7,Feb 2 2023 2:51PM,256557,21,"NBTY Global, Inc.",1,,
8,Feb 2 2023 2:45PM,256556,10,"Digital Brands, LLC",1,,
9,Feb 2 2023 2:40PM,256555,19,ACG North America LLC,1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 3 2023 8:30AM,256565,10,ISDIN Corporation,21,,
1,Feb 3 2023 8:30AM,256564,10,ISDIN Corporation,6,,
2,Feb 3 2023 8:30AM,256568,10,Eywa Pharma Inc.,5,,
3,Feb 3 2023 8:30AM,256569,10,"Nextsource Biotechnology, LLC",1,,
4,Feb 2 2023 4:12PM,256574,16,Sartorius Bioprocess Solutions,2,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 3 2023 8:30AM,256565,10,ISDIN Corporation,21.0,NaN,NaN
1,Feb 3 2023 8:30AM,256564,10,ISDIN Corporation,6.0,NaN,NaN
2,Feb 3 2023 8:30AM,256568,10,Eywa Pharma Inc.,5.0,NaN,NaN
3,Feb 3 2023 8:30AM,256569,10,"Nextsource Biotechnology, LLC",1.0,NaN,NaN
4,Feb 2 2023 4:12PM,256574,16,Sartorius Bioprocess Solutions,2.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 3 2023 8:30AM,256565,10,ISDIN Corporation,21.0,0.0,0.0
1,Feb 3 2023 8:30AM,256564,10,ISDIN Corporation,6.0,0.0,0.0
2,Feb 3 2023 8:30AM,256568,10,Eywa Pharma Inc.,5.0,0.0,0.0
3,Feb 3 2023 8:30AM,256569,10,"Nextsource Biotechnology, LLC",1.0,0.0,0.0
4,Feb 2 2023 4:12PM,256574,16,Sartorius Bioprocess Solutions,2.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2308845,47.0,5.0,3.0
12,769484,1.0,2.0,0.0
15,512941,9.0,7.0,0.0
16,769653,4.0,0.0,0.0
19,1539149,8.0,39.0,12.0
20,256519,1.0,0.0,0.0
21,2052111,7.0,1.0,0.0
22,1026128,4.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2308845,47.0,5.0,3.0
1,12,769484,1.0,2.0,0.0
2,15,512941,9.0,7.0,0.0
3,16,769653,4.0,0.0,0.0
4,19,1539149,8.0,39.0,12.0
5,20,256519,1.0,0.0,0.0
6,21,2052111,7.0,1.0,0.0
7,22,1026128,4.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,47.0,5.0,3.0
1,12,1.0,2.0,0.0
2,15,9.0,7.0,0.0
3,16,4.0,0.0,0.0
4,19,8.0,39.0,12.0
5,20,1.0,0.0,0.0
6,21,7.0,1.0,0.0
7,22,4.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,47.0
1,12,Released,1.0
2,15,Released,9.0
3,16,Released,4.0
4,19,Released,8.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,20,21,22
Status,,,,,,,,
Completed,3.0,0.0,0.0,0.0,12.0,0.0,0.0,0.0
Executing,5.0,2.0,7.0,0.0,39.0,0.0,1.0,0.0
Released,47.0,1.0,9.0,4.0,8.0,1.0,7.0,4.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,20,21,22
0,Completed,3.0,0.0,0.0,0.0,12.0,0.0,0.0,0.0
1,Executing,5.0,2.0,7.0,0.0,39.0,0.0,1.0,0.0
2,Released,47.0,1.0,9.0,4.0,8.0,1.0,7.0,4.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,20,21,22
0,Completed,3.0,0.0,0.0,0.0,12.0,0.0,0.0,0.0
1,Executing,5.0,2.0,7.0,0.0,39.0,0.0,1.0,0.0
2,Released,47.0,1.0,9.0,4.0,8.0,1.0,7.0,4.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()